In [157]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
sns.set()

import sigma_equation as se

In [158]:
subjects = ["soggetto_1","soggetto_2","soggetto_3","soggetto_6","soggetto_7","soggetto_8"]

In [159]:
df_o_dict = {}
for subject in subjects:
    df_o_dict[subject] = pd.read_csv("Datasets/{}_weights_o.csv".format(subject))
    df_o_dict[subject].columns = ["TrRi","RiTr","Plu","Ttra","Ttrb","Tr","MeTr","MoTr","RiRi","MoFo","Mmof","MoRi","MeRi","Ri","Comp"]
    df_o_dict[subject].index = df_o_dict[subject].columns

In [160]:
# plot heatmaps with plotly express
for subject in subjects:
    fig = px.imshow(df_o_dict[subject],
            color_continuous_midpoint=0,
            color_continuous_scale = 'RdBu_r',
            zmin=-5, zmax=5,
            text_auto=".2",
            aspect="Auto",
            title="Original data for {}".format(subject))
    fig.show()

In [161]:
# calculate standard deviations (on axis 1) for each subject
stds = []
for subject in subjects:
    stds.append(df_o_dict[subject].std(axis=1))
stds = pd.DataFrame(stds)
stds.index = subjects
stds.columns = df_o_dict[subject].columns
stds

,TrRi,RiTr,Plu,Ttra,Ttrb,Tr,MeTr,MoTr,RiRi,MoFo,Mmof,MoRi,MeRi,Ri,Comp
soggetto_1,0.220196,0.223144,0.333889,0.437770,0.213160,0.228997,0.284814,0.177968,0.236333,0.214411,0.185073,0.297189,0.253959,0.377430,0.252651
soggetto_2,0.492646,0.363330,0.517538,0.312095,0.385892,0.122184,0.389117,0.385330,0.465440,0.258724,0.403519,0.264124,0.383842,0.433798,0.425059
soggetto_3,1.290731,0.798684,0.966400,0.715229,1.453136,0.661781,0.650314,0.544598,0.330313,0.710032,0.812042,0.894647,0.405209,0.501343,1.096286
soggetto_6,0.634487,0.959705,0.746840,1.216884,0.605079,0.922659,0.977046,1.284632,0.683250,0.809403,1.079089,0.853820,0.713823,1.024133,0.730252
soggetto_7,0.562738,0.313070,1.146880,0.845131,0.751379,0.673072,1.289125,0.797708,2.019221,1.006056,0.965304,0.732277,0.971913,0.849081,0.537363
soggetto_8,2.040547,1.172403,1.327796,0.890147,0.822927,1.342752,2.026490,0.988268,1.101534,0.706036,1.042398,1.864858,0.768913,1.271345,0.800889


In [162]:
# plot stds with barplot with plotly express
fig = px.bar(stds, barmode="group", title="Standard deviations for each subject")
# use different colors for each subject
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b"]
# ...
fig.show()

In [163]:
# for each subject's dataframe create two numpy arrays, one with positive values and one for negatives
df_ecc_dict = {}
df_ini_dict = {}
for subject in subjects:
    df_ecc_dict[subject] = df_o_dict[subject].where(df_o_dict[subject] > 0)
    df_ini_dict[subject] = df_o_dict[subject].where(df_o_dict[subject] < 0)
    # put the other values to zeros
    df_ecc_dict[subject] = df_ecc_dict[subject].fillna(0)
    df_ini_dict[subject] = df_ini_dict[subject].fillna(0)
    # transform the negative values of df_n_dict[subject] in positives
    df_ini_dict[subject] = df_ini_dict[subject] * (-1)
    df_ini_dict[subject].replace(-0.0, 0.0)
    # trasform each dataframe in a stochastic matrix by column (the sum of each column must be equal to 1)
    df_ecc_dict[subject] = df_ecc_dict[subject].div(df_ecc_dict[subject].sum(axis=0), axis=1)
    df_ini_dict[subject] = df_ini_dict[subject].div(df_ini_dict[subject].sum(axis=0), axis=1)

In [164]:
# create dicts with numpy instead dataframes
np_ecc_dict = {}
np_ini_dict = {}
for subject in subjects:
    np_ecc_dict[subject] = df_ecc_dict[subject].to_numpy()
    np_ini_dict[subject] = df_ini_dict[subject].to_numpy()

In [165]:
df_evolve_dict_out = {}
df_evolve_dict_in = {}
for subject in subjects:
    df_evolve_dict_out[subject] = pd.DataFrame(columns=df_o_dict[subject].columns)
    df_evolve_dict_in[subject] = pd.DataFrame(columns=df_o_dict[subject].columns)
for subject in subjects:
    for i in range(0,50):
        np_ecc_dict[subject] = np.matmul(np_ecc_dict[subject], np_ecc_dict[subject])
        np_ini_dict[subject] = np.matmul(np_ini_dict[subject], np_ini_dict[subject])
        np_activations_out = np.sum(np_ecc_dict[subject], axis=1) - np.sum(np_ini_dict[subject], axis=1) # two markov chains that interfere with each other
        np_activations_out = np_activations_out.reshape(15,1)
        df_evolve_dict_out[subject] = pd.concat(
            [
                df_evolve_dict_out[subject],
                pd.DataFrame(np_activations_out.T, columns = df_evolve_dict_out[subject].columns)
            ]
            , axis = 0)
        np_activations_in = np.sum(np_ecc_dict[subject], axis=0) - np.sum(np_ini_dict[subject], axis=0)
        np_activations_in = np_activations_in.reshape(15,1)
        df_evolve_dict_in[subject] = pd.concat(
            [
                df_evolve_dict_in[subject],
                pd.DataFrame(np_activations_in.T, columns = df_evolve_dict_in[subject].columns)
            ]
            , axis = 0)
    

In [166]:
# for each subject's dataframe adjust the index to start from 0
for subject in subjects:
    df_evolve_dict_out[subject].index = range(0,df_evolve_dict_out[subject].shape[0])
    df_evolve_dict_in[subject].index = range(0,df_evolve_dict_in[subject].shape[0])

In [167]:
# create dataframes for each subject with the evolution of the matrix
df_steady_dict = {}
for subject in subjects:
    df_steady_dict[subject] = pd.DataFrame(np_ecc_dict[subject] - np_ini_dict[subject], columns=df_o_dict[subject].columns, index=df_o_dict[subject].columns)

In [168]:
df_steady_out = pd.DataFrame(columns=subjects, index=df_o_dict[subject].columns)
df_steady_in = pd.DataFrame(columns=subjects, index=df_o_dict[subject].columns)
for subject in subjects:
    df_steady_out[subject] = df_steady_dict[subject].sum(axis = 1)
    df_steady_in[subject] = df_steady_dict[subject].sum(axis = 0)

In [169]:
df_steady_out

,soggetto_1,soggetto_2,soggetto_3,soggetto_6,soggetto_7,soggetto_8
TrRi,-3.653902,0.889522,-0.695046,0.147795,-0.680561,-1.428025
RiTr,-1.085590,1.182842,-0.535501,0.166360,0.848230,-0.551518
Plu,0.430688,1.904225,-0.233821,-0.141018,-1.629154,1.112462
Ttra,1.511440,-0.376032,0.256865,-1.714616,0.240710,0.014481
Ttrb,1.581942,-1.105314,0.379338,-0.173209,-1.702613,-0.164470
Tr,1.193570,-3.210715,-0.828790,1.448480,0.171163,0.280844
MeTr,0.113789,0.307309,-1.671061,1.112112,0.071179,1.117429
MoTr,-0.925576,0.057244,0.732576,-1.394138,0.323031,-0.048339
RiRi,-1.440676,0.784019,2.231494,-0.412701,0.040740,-0.943371
MoFo,2.224161,0.985689,0.315306,-0.019959,1.426594,-0.077513


In [179]:
df_steady_out.mean(axis=0), df_steady_out.std(axis=0)

(soggetto_1    0.021889
 soggetto_2   -0.038652
 soggetto_3   -0.031826
 soggetto_6   -0.070046
 soggetto_7   -0.032063
 soggetto_8   -0.021056
 dtype: float64,
 soggetto_1    1.594039
 soggetto_2    1.257323
 soggetto_3    0.946273
 soggetto_6    0.824227
 soggetto_7    0.906985
 soggetto_8    0.785711
 dtype: float64)

In [170]:
df_steady_in

,soggetto_1,soggetto_2,soggetto_3,soggetto_6,soggetto_7,soggetto_8
TrRi,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
RiTr,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
Plu,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
Ttra,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
Ttrb,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
Tr,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
MeTr,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
MoTr,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
RiRi,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056
MoFo,0.021889,-0.038652,-0.031826,-0.070046,-0.032063,-0.021056


In [180]:
df_steady_in.mean(axis=0), df_steady_in.std(axis=0)

(soggetto_1    0.021889
 soggetto_2   -0.038652
 soggetto_3   -0.031826
 soggetto_6   -0.070046
 soggetto_7   -0.032063
 soggetto_8   -0.021056
 dtype: float64,
 soggetto_1    2.338432e-16
 soggetto_2    2.126859e-16
 soggetto_3    3.917513e-16
 soggetto_6    2.464180e-16
 soggetto_7    2.914748e-16
 soggetto_8    1.718765e-16
 dtype: float64)

In [171]:
# plot the df_steady_out and df_steady_in with plotly express
fig = px.bar(df_steady_out.T, barmode="group", title="Steady state out")
fig.show()
fig = px.bar(df_steady_in.T, barmode="group", title="Steady state in")
fig.show()

In [172]:
# plot lineplots with plotly express for each subject
for subject in subjects:
    fig = px.line(df_evolve_dict_in[subject], title="Evolution of activations IN for {}".format(subject))
    fig.show()

In [173]:
# plot lineplots with plotly express for each subject
for subject in subjects:
    fig = px.line(df_evolve_dict_out[subject], title="Evolution of activations OUT for {}".format(subject))
    fig.show()